## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from Signals_Processing import *
from sklearn.ensemble import VotingClassifier
import joblib
from sklearn import metrics
import matplotlib.pyplot as plt

## Load Dataframes

In [3]:
dataframe = pd.read_csv('.\Breathing Rate\Results\sr_dataframe.csv', sep=",", index_col=False)
dataframe = dataframe.drop(["Unnamed: 0","RRV_SD2","RRV_SD2SD1","RRV_ApEn"],axis=1)
dataframe

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MCVBB,RRV_nn50,RRV_nn20,RRV_pNN50,RRV_pNN20,RRV_SD1,Arousal,Valence,User,Condition
0,91.053187,84.497321,109.078360,4.903607,0.660701,0.550063,0.710082,32.4977,15.9249,3,...,0.250260,11.0,14.0,64.705882,82.352941,190.854171,Medium,Medium,P1,VR_1
1,90.756509,86.944117,93.736626,1.608937,0.661319,0.640091,0.690098,11.8049,9.1957,0,...,0.313178,11.0,12.0,84.615385,92.307692,130.662458,High,High,P1,VR_2
2,90.369801,85.708316,93.743471,1.711707,0.664178,0.640045,0.700049,12.6285,8.9026,0,...,0.236945,19.0,21.0,79.166667,87.500000,167.888150,Medium,Medium,P1,VR_3
3,91.392440,86.947491,93.740264,1.766946,0.656758,0.640066,0.690072,12.8779,10.1768,0,...,0.423092,14.0,15.0,82.352941,88.235294,170.125977,Low,High,P1,VR_4
4,90.339634,86.946247,96.762759,1.498535,0.664339,0.620073,0.690082,10.8007,10.9638,1,...,0.328706,9.0,12.0,60.000000,80.000000,137.894101,Low,High,P1,VR_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,70.119985,23.712030,249.964321,40.052733,1.043649,0.240034,2.530361,465.7141,421.9698,19,...,0.142901,4.0,6.0,28.571429,42.857143,69.907952,Low,Medium,P9,VR_4
348,76.649369,58.814151,222.186791,36.086855,0.857012,0.270043,1.020163,163.4676,196.2183,19,...,0.233311,7.0,7.0,70.000000,70.000000,118.678721,Low,Medium,P9,VR_5
349,66.322104,59.396514,74.988099,3.410516,0.907029,0.800127,1.010160,45.8703,49.4379,21,...,0.104115,4.0,11.0,28.571429,78.571429,129.855837,Low,Medium,P9,VR_6
350,86.065374,62.490396,260.829481,49.601103,0.809939,0.230035,0.960148,200.1834,222.3759,22,...,0.056480,1.0,10.0,5.882353,58.823529,20.636083,Low,Medium,P9,VR_7


In [4]:
df_baseline = pd.read_csv('.\Breathing Rate\Results\df_baseline.csv', sep=";", index_col=False)
df_baseline

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_HF,RRV_SD1,User
0,91.739392,86.953623,103.444828,2.245824,0.654405,0.580019,0.690023,15.4795,11.0191,4,...,372.5,46.7019,0.125374,51.0,33.0,50.000000,77.272727,0.112523,132.476615,P1
1,69.560333,63.827660,81.078378,3.209863,0.864360,0.740025,0.940031,39.0766,34.9855,64,...,462.5,46.7019,0.100977,37.0,16.0,26.666667,61.666667,0.148027,79.259154,P10
2,89.823768,77.919481,107.139286,5.393923,0.670336,0.560019,0.770026,39.4640,19.5575,8,...,441.0,100.8168,0.228610,51.0,34.0,57.627119,86.440678,0.140153,103.814933,P11
3,68.992448,63.827660,81.078378,2.861169,0.871111,0.740025,0.940031,35.0408,24.6032,19,...,408.0,84.5082,0.207128,51.0,34.0,52.307692,78.461538,0.051048,139.386705,P12
4,55.941228,46.510078,67.413483,5.015857,1.081135,0.890030,1.290043,96.1119,79.6572,180,...,680.5,131.2101,0.192814,31.0,22.0,55.000000,77.500000,0.027679,73.984739,P13
5,63.204438,58.250485,74.997500,2.700314,0.950990,0.800027,1.030034,39.6287,30.7189,41,...,267.0,28.1694,0.105503,48.0,27.0,28.421053,50.526316,0.094706,96.751455,P14
6,63.157412,50.845763,75.946835,4.314817,0.954423,0.790026,1.180039,64.9401,50.2295,116,...,419.0,53.3736,0.127383,38.0,20.0,30.769231,58.461538,0.095545,54.794719,P15
7,78.893679,64.513978,95.234921,6.287741,0.765389,0.630021,0.930031,61.3838,54.5838,146,...,465.0,71.9061,0.154637,46.0,21.0,33.870968,74.193548,0.052039,60.897643,P16
8,65.728696,55.044037,90.906061,4.490089,0.916937,0.660022,1.090036,60.4751,59.1230,133,...,340.0,20.7564,0.061048,29.0,7.0,8.433735,34.939759,0.037646,46.963795,P17
9,100.875881,85.711429,124.995833,5.412872,0.596460,0.480016,0.700023,31.2184,28.5777,64,...,417.0,36.3237,0.087107,32.0,17.0,27.419355,51.612903,0.046736,199.993582,P18


In [5]:
columns = dataframe.columns[:(len(dataframe.columns)-4)]

In [18]:
users = np.unique(dataframe["User"])
users

array(['P1', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17',
       'P18', 'P19', 'P2', 'P20', 'P21', 'P22', 'P23', 'P24', 'P25',
       'P26', 'P27', 'P28', 'P29', 'P3', 'P30', 'P4', 'P5', 'P6', 'P7',
       'P8', 'P9'], dtype=object)

In [24]:
full_dataframe = pd.DataFrame(columns=columns)
for user in users:
    print("User = " + user)
    baseline = df_baseline[df_baseline["User"] == user]
    for index, row in dataframe[dataframe["User"] == user].iterrows():
        full_dataframe = pd.concat(
            [full_dataframe, row[columns]-baseline[columns]], ignore_index=True)

User = P1
User = P10
User = P11
User = P12
User = P13
User = P14
User = P15
User = P16
User = P17
User = P18
User = P19
User = P2
User = P20
User = P21
User = P22
User = P23
User = P24
User = P25
User = P26
User = P27
User = P28
User = P29
User = P3
User = P30
User = P4
User = P5
User = P6
User = P7
User = P8
User = P9


In [25]:
full_dataframe["Users"] = dataframe["User"]
full_dataframe["Arousal"] = dataframe["Arousal"]
full_dataframe["Valence"] = dataframe["Valence"]
full_dataframe["Condition"] = dataframe["Condition"]

In [ ]:
dataframe["Condition"]

In [26]:
full_dataframe

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MedianBB,RRV_MadBB,RRV_MCVBB,RRV_pNN50,RRV_pNN20,RRV_SD1,Users,Arousal,Valence,Condition
0,-0.686205,-2.456302,5.633532,2.657783,0.006296,-0.029956,0.020059,17.0182,4.9058,-1,...,18.5,51.1497,0.124886,14.705882,5.080214,58.377556,P1,Medium,Medium,VR_1
1,-0.982883,-0.009506,-9.708201,-0.636887,0.006914,0.060072,0.000075,-3.6746,-1.8234,-4,...,72.5,92.6625,0.187804,34.615385,15.034965,-1.814157,P1,High,High,VR_2
2,-1.369591,-1.245307,-9.701357,-0.534117,0.009773,0.060025,0.010026,-2.851,-2.1165,-4,...,175.0,83.0256,0.111571,29.166667,10.227273,35.411534,P1,Medium,Medium,VR_3
3,-0.346952,-0.006132,-9.704564,-0.478878,0.002353,0.060047,0.000049,-2.6016,-0.8423,-4,...,44.5,129.7275,0.297718,32.352941,10.962567,37.649361,P1,Low,High,VR_4
4,-1.399758,-0.007376,-6.682069,-0.747289,0.009934,0.040054,0.000059,-4.6788,-0.0553,-3,...,60.5,95.6277,0.203332,10.0,2.727273,5.417485,P1,Low,High,VR_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,4.731544,-35.109538,163.010698,36.659019,0.123743,-0.449989,1.510327,420.8181,375.528,-85,...,-51.0,-19.2738,-0.031295,-18.487395,-36.554622,-14.408991,P9,Low,Medium,VR_4
348,11.260928,-0.007418,135.233168,32.693141,-0.062895,-0.41998,0.000129,118.5716,149.7765,-85,...,65.0,37.8063,0.059115,22.941176,-9.411765,34.361778,P9,Low,Medium,VR_5
349,0.933663,0.574945,-11.965524,0.016801,-0.012877,0.110104,-0.009874,0.9743,2.9961,-83,...,-27.0,-29.652,-0.070081,-18.487395,-0.840336,45.538894,P9,Low,Medium,VR_6
350,20.676933,3.668828,173.875858,46.207388,-0.109967,-0.459988,-0.059886,155.2874,175.9341,-82,...,-68.0,-48.9258,-0.117716,-41.176471,-20.588235,-63.68086,P9,Low,Medium,VR_7


In [28]:
full_dataframe.to_csv('.\Breathing Rate\Results\sr_full_dataframe.csv')

In [ ]:
full_dataframe[full_dataframe["Users"] == "P3"]

## Load Full Dataframe

In [8]:
full_dataframe = pd.read_csv(r'.\Breathing Rate\Results\full_dataframe.csv', sep=";", index_col=False)
full_dataframe = full_dataframe.drop("RRV_HF",axis=1)
full_dataframe

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MCVBB,RRV_nn20,RRV_nn50,RRV_pNN50,RRV_pNN20,RRV_SD1,Users,Arousal,Valence,Condition
0,-0.181209,1.266965,-6.686763,-0.469262,0.001159,0.040084,-0.009910,-2.9630,-3.5900,-4,...,0.083627,-43.0,-25.0,38.888889,11.616162,43.757984,P1,Low,Medium,Baseline_1
1,-0.188672,1.266965,-6.686763,-0.477884,0.001210,0.040084,-0.009910,-3.0243,-3.5558,-4,...,0.077095,-43.0,-25.0,38.888889,11.616162,43.214164,P1,Low,Medium,Baseline_2
2,-0.270225,1.266965,-6.686763,-0.564435,0.001772,0.040084,-0.009910,-3.6152,-3.4801,-4,...,0.079702,-43.0,-25.0,30.000000,2.727273,31.694958,P1,Low,Medium,Baseline_3
3,-0.318858,1.266965,-6.686763,-0.611762,0.002109,0.040084,-0.009910,-3.9390,-3.4801,-4,...,0.078021,-43.0,-25.0,30.000000,2.727273,31.549678,P1,Low,Medium,Baseline_4
4,-0.417667,1.266965,-8.222605,-0.766110,0.002783,0.050086,-0.009910,-4.9627,-3.4050,-4,...,0.083064,-43.0,-25.0,30.000000,2.727273,32.236814,P1,Low,Medium,Baseline_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38894,0.938824,2.392717,-12.891895,-0.687346,-0.013818,0.120112,-0.039871,-8.5015,-4.3087,-87,...,-0.133202,-50.0,-32.0,-47.058824,-54.411765,-69.345065,P9,Low,Medium,VR_490
38895,0.884367,2.392717,-15.536957,-0.811299,-0.013193,0.150117,-0.039871,-9.8358,-3.2848,-86,...,-0.124474,-50.0,-32.0,-47.058824,-54.411765,-69.514755,P9,Low,Medium,VR_491
38896,15.557454,2.392717,198.713043,44.832734,-0.075220,-0.479988,-0.039871,135.3543,68.6136,-85,...,-0.128856,-50.0,-32.0,-47.058824,-52.745098,-70.598823,P9,Low,Medium,VR_492
38897,9.634481,2.392717,198.713043,36.036045,-0.050112,-0.479988,-0.039871,95.7147,70.9675,-84,...,-0.133390,-51.0,-32.0,-47.058824,-59.411765,-71.222086,P9,Low,Medium,VR_493


In [9]:
sr_dataframe = pd.read_csv(r'.\Breathing Rate\Results\sr_dataframe.csv', sep=",", index_col=False)
sr_dataframe = sr_dataframe.drop(["Unnamed: 0","RRV_SD2","RRV_SD2SD1","RRV_ApEn"],axis=1)
sr_dataframe

,Avg HR,Min HR,Max HR,SD,AVG RR,Minimum RR,Maximum RR,SDNN,RMSSD,NN50,...,RRV_MCVBB,RRV_nn50,RRV_nn20,RRV_pNN50,RRV_pNN20,RRV_SD1,Arousal,Valence,User,Condition
0,91.053187,84.497321,109.078360,4.903607,0.660701,0.550063,0.710082,32.4977,15.9249,3,...,0.250260,11.0,14.0,64.705882,82.352941,190.854171,Medium,Medium,P1,VR_1
1,90.756509,86.944117,93.736626,1.608937,0.661319,0.640091,0.690098,11.8049,9.1957,0,...,0.313178,11.0,12.0,84.615385,92.307692,130.662458,High,High,P1,VR_2
2,90.369801,85.708316,93.743471,1.711707,0.664178,0.640045,0.700049,12.6285,8.9026,0,...,0.236945,19.0,21.0,79.166667,87.500000,167.888150,Medium,Medium,P1,VR_3
3,91.392440,86.947491,93.740264,1.766946,0.656758,0.640066,0.690072,12.8779,10.1768,0,...,0.423092,14.0,15.0,82.352941,88.235294,170.125977,Low,High,P1,VR_4
4,90.339634,86.946247,96.762759,1.498535,0.664339,0.620073,0.690082,10.8007,10.9638,1,...,0.328706,9.0,12.0,60.000000,80.000000,137.894101,Low,High,P1,VR_5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,70.119985,23.712030,249.964321,40.052733,1.043649,0.240034,2.530361,465.7141,421.9698,19,...,0.142901,4.0,6.0,28.571429,42.857143,69.907952,Low,Medium,P9,VR_4
348,76.649369,58.814151,222.186791,36.086855,0.857012,0.270043,1.020163,163.4676,196.2183,19,...,0.233311,7.0,7.0,70.000000,70.000000,118.678721,Low,Medium,P9,VR_5
349,66.322104,59.396514,74.988099,3.410516,0.907029,0.800127,1.010160,45.8703,49.4379,21,...,0.104115,4.0,11.0,28.571429,78.571429,129.855837,Low,Medium,P9,VR_6
350,86.065374,62.490396,260.829481,49.601103,0.809939,0.230035,0.960148,200.1834,222.3759,22,...,0.056480,1.0,10.0,5.882353,58.823529,20.636083,Low,Medium,P9,VR_7


## Arousal

In [10]:
columns = full_dataframe.columns[:len(full_dataframe.columns)-4]
columns

Index(['Avg HR', 'Min HR', 'Max HR', 'SD', 'AVG RR', 'Minimum RR',
       'Maximum RR', 'SDNN', 'RMSSD', 'NN50', 'pNN50', 'NN20', 'pNN20', 'SD1',
       'SD2', 'SD2/SD1', 'HRV VLF Power', 'HRV LF Power', 'HRV HF Power',
       'HRV Total Power', 'HRV LF (nu)', 'HRV HF (nu)', 'HRV LF/HF',
       'Phasic_AVG', 'Phasic_MAX', 'Phasic_MIN', 'Phasic_STD', 'Tonic_AVG',
       'Tonic_MAX', 'Tonic_MIN', 'Tonic_STD', 'SCR_Amp_AVG', 'SCR_Amp_MAX',
       'SCR_Amp_MIN', 'SCR_Amp_STD', 'SCR_Rt_AVG', 'SCR_Rt_MAX', 'SCR_Rt_MIN',
       'SCR_Rt_STD', 'SCR_Rect_AVG', 'SCR_Rect_MAX', 'SCR_Rect_MIN',
       'SCR_Rect_STD', 'VLF Power', 'LF Power', 'HF Power', 'Total Power',
       'LF (nu)', 'HF (nu)', 'Mean_RSP_Rate', 'Minimum_RSP_Rate',
       'Maximum_RSP_Rate', 'STD_RSP_Rate', 'Mean_RSP_Amplitude',
       'Minimum_RSP_Amplitude', 'Maximum_RSP_Amplitude', 'STD_RSP_Amplitude',
       'RRV_RMSSD', 'RRV_MeanBB', 'RRV_SDBB', 'RRV_SDSD', 'RRV_CVBB',
       'RRV_CVSD', 'RRV_MedianBB', 'RRV_MadBB', 'RRV_MCVB

## GridSearchCV

In [8]:
param_grids = {
    "KNN": {
        'model__n_neighbors': list(range(1,5)), 
        'model__weights': ['uniform', 'distance'],
        'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    "SVM": {
        'model__C': [0.1, 1, 10], 
        'model__kernel': ['linear', 'rbf'],
        'model__gamma': ["scale","auto"],
        'model__random_state': list(range(0,50,2))
    },
    "Bayes": {},  # No parameters for GaussianNB
    "Logistic Regression": {
        'model__C': [0.1, 1, 10], 
        'model__solver': ['lbfgs','newton-cg','sag','saga'],
        'model__multi_class': ["multinomial"],
        'model__random_state': list(range(0,50,5))
    },
    "Random Forest": {
        'model__n_estimators': [50,100,150], 
        'model__criterion': ["gini","entropy","log_loss"],
        'model__random_state': list(range(0,50,5))
    },
    "NN": { 
        'model__activation': ['identity', 'logistic', 'tanh', 'relu'],
        'model__solver': ['lbfgs', 'sgd', 'adam'],
        'model__random_state': list(range(0,50,5))
    }
}

In [10]:
from sklearn.pipeline import Pipeline
warnings.filterwarnings("ignore")

users_dict = {}
grouped_users = full_dataframe.groupby('Users')



for user_id, user_data in grouped_users:
    
    results = []
    
    print("User = {}".format(user_id))

    y_true, y_predict, model_accuracy, samples = [], [], [], []
    user_accuracy, accuracy_scores = {}, {}

    for model_name, model in models:
        model_y_true, model_y_predict = [], []
        
        print(f"Model = {model_name}")

        # Separate training and testing data
        training_data = user_data[user_data["Condition"].str.contains(
            "Baseline") | user_data["Condition"].str.contains("PreStroop")]
        testing_data = user_data[user_data["Condition"].str.contains(
            "VR")] #| user_data["Condition"].str.contains("PostStroop")]

        X_train = np.array(training_data[columns])
        X_test = np.array(testing_data[columns])
        Y_train = np.array(training_data["Arousal"])
        Y_test = np.array(testing_data["Arousal"])

        # Create the pipeline
        pipeline = Pipeline([
            # Impute missing values
            ('imputer', SimpleImputer(missing_values=np.nan, strategy="mean")),
            ('scaler', StandardScaler()),  # Standardize the features
            ('feature_selection', feature_selection.RFE(ensemble.RandomForestClassifier(
                random_state=42), step=1)),  # Recursive feature elimination
            ('model', model)  # The model, which will be set in GridSearch
        ])

        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})

        # Grid search for hyperparameter tuning
        if param_grid:
            grid_search = GridSearchCV(
                pipeline, param_grid, cv=5, n_jobs=6, scoring='accuracy')
            grid_search.fit(X_train, Y_train.ravel())
            best_pipeline = grid_search.best_estimator_
            best_params = grid_search.best_params_
            print(
                f"Best parameters for {model_name}: {grid_search.best_params_}")
        else:
            # Fit the pipeline without grid search if no parameters
            pipeline.fit(X_train, Y_train.ravel())
            best_pipeline = pipeline
            best_params = {}

        # Evaluate the best model on the test data
        y_true.extend(Y_test)
        y_predict.extend(best_pipeline.predict(X_test))
        accuracy = metrics.accuracy_score(y_true, y_predict)
        
        print(f"Accuracy = {accuracy*100:.2f}%")
        
        # Append results for this user and model
        results.append({
            'User': user_id,
            'Model': model_name,
            'Best Parameters': str(best_params),
            'Test Accuracy': accuracy
        })
        
    # Save intermediate results to avoid memory overflow
    pd.DataFrame(results).to_csv(f'./user_results/results_{user_id}.csv', index=False)
    del X_train, X_test, Y_train, Y_test, best_pipeline, y_true, y_predict  # Clear memory

# After all users, consolidate all CSVs into a final result
final_results_df = pd.concat([pd.read_csv(f'./user_results/results_{user_id}.csv') for user_id in grouped_users.groups.keys()])
final_results_df.to_csv('model_results.csv', index=False)

User = P1
Model = KNN
Best parameters for KNN: {'model__algorithm': 'auto', 'model__n_neighbors': 2, 'model__weights': 'uniform'}
Accuracy = 49.07%
Model = SVM
Best parameters for SVM: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf', 'model__random_state': 0}
Accuracy = 51.08%
Model = Bayes
Accuracy = 49.95%
Model = Logistic Regression
Best parameters for Logistic Regression: {'model__C': 0.1, 'model__multi_class': 'multinomial', 'model__random_state': 0, 'model__solver': 'lbfgs'}
Accuracy = 48.30%
Model = Random Forest
Best parameters for Random Forest: {'model__criterion': 'gini', 'model__n_estimators': 50, 'model__random_state': 0}
Accuracy = 47.06%
Model = NN
Best parameters for NN: {'model__activation': 'relu', 'model__random_state': 5, 'model__solver': 'sgd'}
Accuracy = 46.26%
User = P10
Model = KNN
Best parameters for KNN: {'model__algorithm': 'auto', 'model__n_neighbors': 1, 'model__weights': 'uniform'}
Accuracy = 63.43%
Model = SVM
Best parameters for SVM: {'m

Best parameters for SVM: {'model__C': 0.1, 'model__gamma': 'scale', 'model__kernel': 'linear', 'model__random_state': 0}
Accuracy = 86.09%
Model = Bayes
Accuracy = 86.95%
Model = Logistic Regression
Best parameters for Logistic Regression: {'model__C': 1, 'model__multi_class': 'multinomial', 'model__random_state': 0, 'model__solver': 'lbfgs'}
Accuracy = 86.72%
Model = Random Forest
Best parameters for Random Forest: {'model__criterion': 'gini', 'model__n_estimators': 100, 'model__random_state': 40}
Accuracy = 78.98%
Model = NN
Best parameters for NN: {'model__activation': 'tanh', 'model__random_state': 40, 'model__solver': 'lbfgs'}
Accuracy = 77.59%
User = P2
Model = KNN
Best parameters for KNN: {'model__algorithm': 'auto', 'model__n_neighbors': 4, 'model__weights': 'uniform'}
Accuracy = 97.62%
Model = SVM
Best parameters for SVM: {'model__C': 0.1, 'model__gamma': 'scale', 'model__kernel': 'linear', 'model__random_state': 0}
Accuracy = 97.75%
Model = Bayes
Accuracy = 97.48%
Model = Log

Accuracy = 56.66%
Model = Logistic Regression
Best parameters for Logistic Regression: {'model__C': 10, 'model__multi_class': 'multinomial', 'model__random_state': 5, 'model__solver': 'sag'}
Accuracy = 55.05%
Model = Random Forest
Best parameters for Random Forest: {'model__criterion': 'entropy', 'model__n_estimators': 150, 'model__random_state': 5}
Accuracy = 53.03%
Model = NN
Best parameters for NN: {'model__activation': 'relu', 'model__random_state': 45, 'model__solver': 'sgd'}
Accuracy = 52.88%
User = P29
Model = KNN
Best parameters for KNN: {'model__algorithm': 'auto', 'model__n_neighbors': 1, 'model__weights': 'uniform'}
Accuracy = 55.32%
Model = SVM
Best parameters for SVM: {'model__C': 0.1, 'model__gamma': 'scale', 'model__kernel': 'rbf', 'model__random_state': 0}
Accuracy = 50.58%
Model = Bayes
Accuracy = 33.77%
Model = Logistic Regression
Best parameters for Logistic Regression: {'model__C': 0.1, 'model__multi_class': 'multinomial', 'model__random_state': 0, 'model__solver': 

In [ ]:
final_results_df

## Train Models

### Models

In [27]:
from sklearn.base import clone
def is_incremental_model(model):
    """Check if the model supports incremental learning (i.e., has `partial_fit` method)."""
    return hasattr(model, "partial_fit")

In [10]:
models=[]
# models.append(("KNN",neighbors.KNeighborsClassifier()))
models.append(("SVM",svm.SVC(probability=True)))
models.append(("Bayes", naive_bayes.GaussianNB()))
models.append(("Logistic Regression", LogisticRegression()))
models.append(("Random Forest",ensemble.RandomForestClassifier()))
models.append(("NN",MLPClassifier()))

### Select Best 2 models to create Voting Classifier

In [13]:
import warnings, joblib
warnings.filterwarnings("ignore")
users_dict = {}

grouped_users = full_dataframe.groupby('Users')

for user_id, user_data in grouped_users:
    
    results = []
    
    # Load saved results for this user
    csv_file = f'./user_results/results_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user

    print("User = {}".format(user_id))

    y_true, y_predict, model_accuracy, samples = [], [], [], []

    user_accuracy, accuracy_scores = {}, {}

    training_data = user_data[user_data["Condition"].str.contains(
            "Baseline") | user_data["Condition"].str.contains("PreStroop")]
    testing_data = sr_dataframe[sr_dataframe["User"] == user_id]

    X_train = np.array(training_data[columns])
    X_test = np.array(testing_data[columns])
    Y_train = np.array(training_data["Arousal"])
    Y_test = np.array(testing_data["Arousal"])

    accuracy_scores = {}
    pipelines = []
    
    for model_name,model in models:

        print(f'Model = {model_name}')

        user_results = results_df[(results_df['Model'] == model_name)]
        
        best_params = eval(user_results['Best Parameters'].values[0])  # Convert string back to dictionary
        best_params = {key.replace('model__', ''): value for key, value in best_params.items()} #remove 'model__' prefix
        
        model.set_params(**best_params)

        # Create the pipeline
        pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(missing_values=np.nan, strategy="mean")),
                ("scaler", StandardScaler()),
                (
                    "feature_selection",
                    feature_selection.RFE(
                        ensemble.RandomForestClassifier(random_state=42), step=1
                    ),
                ),
                (
                    "model",
                    model
                ),  
            ]
        )

        pipeline.fit(X_train, Y_train.ravel())
    
        model_y_predict = pipeline.predict(X_test)
    
        accuracy = metrics.accuracy_score(Y_test, model_y_predict)
    
        # Store the accuracy and pipeline
        accuracy_scores[model_name] = accuracy * 100
        pipelines.append((model_name, pipeline))

    # Select the two best pipelines by accuracy
    sorted_models = sorted(accuracy_scores.items(), key=lambda item: item[1], reverse=True)
    best_two_model_names = [sorted_models[0][0], sorted_models[1][0]]
    best_two_pipelines = [pipeline for model_name, pipeline in pipelines if model_name in [sorted_models[0][0], sorted_models[1][0]]]

    # Save the best two pipelines to files
    for i, pipeline in enumerate(best_two_pipelines):
        model_name = best_two_model_names[i]  # Get the model name from the best model names
        joblib.dump(pipeline, f"./user_models/{user_id}_best_pipeline_{i+1}_{model_name}.pkl")
    
    
    print(best_two_pipelines)
    
   # Create the Voting Classifier with the entire pipelines as estimators
    estimators = [(best_two_model_names[i], best_two_pipelines[i]) for i in range(2)]

    voting_clf = VotingClassifier(estimators=estimators, voting='soft', n_jobs=-1)

    voting_clf.fit(X_train,Y_train.ravel())

    joblib.dump(voting_clf, f"./user_models/model_{user_id}.pkl")

    y_true.extend(Y_test)
    y_predict.extend(voting_clf.predict(X_test))

    print(f'Len Testing data = {len(testing_data)}')
    print(f'Len Y_true = {len(y_true)}')
    print(f'Len Y_predict = {len(y_predict)}')

    accuracy = metrics.accuracy_score(y_true, y_predict)*100

    model_accuracy.append(accuracy)

    results.append({
        "User": user_id,
        "True labels": y_true,
        "Predictions": y_predict,
        "Accuracy": accuracy
    })

    pd.DataFrame(results).to_csv(f'./voting_results/results_{user_id}.csv', index=False)

User = P1
Model = SVM
Model = Bayes
Model = Logistic Regression
Model = Random Forest
Model = NN
[Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('feature_selection',
                 RFE(estimator=RandomForestClassifier(random_state=42))),
                ('model', SVC(C=10, probability=True, random_state=0))]), Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('feature_selection',
                 RFE(estimator=RandomForestClassifier(random_state=42))),
                ('model', MLPClassifier(random_state=5, solver='sgd'))])]
Len Testing data = 8
Len Y_true = 8
Len Y_predict = 8
User = P10
Model = SVM
Model = Bayes
Model = Logistic Regression
Model = Random Forest
Model = NN
[Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler()),
                ('feature_selection',
                 RFE(estimator=RandomForestClassifier(random_state=42))),
                ('model',
           

In [14]:
results

[{'User': 'P9',
  'True labels': ['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low'],
  'Predictions': ['Low',
   'Medium',
   'Low',
   'Medium',
   'Low',
   'Low',
   'Low',
   'Low'],
  'Accuracy': 75.0}]

### Accuracy from Self-Reports

In [29]:
grouped_users = full_dataframe.groupby('Users')

dataframes=[]

for user_id, user_data in grouped_users:
    # Load saved results for this user
    csv_file = f'./voting_results/results_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user
    dataframes.append(results_df)

df_users = pd.concat(dataframes,ignore_index=True)

In [30]:
df_users

,User,True labels,Predictions,Accuracy
0,P1,"['Medium', 'High', 'Medium', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",50.000000
1,P10,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Medium', 'High', 'Low', 'Low', 'High...",69.230769
2,P11,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...",83.333333
3,P12,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['High', 'High', 'High', 'High', 'High', 'High...",0.000000
4,P13,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...",83.333333
5,P14,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...",41.666667
6,P15,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...",91.666667
7,P16,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",100.000000
8,P17,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",0.000000
9,P18,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...",8.333333


In [31]:
grouped_users = df_users.groupby('User')

correct_predictions = {}

for user_id, user_data in grouped_users:
    true_labels = eval(user_data["True labels"].values[0])
    prediction = eval(user_data["Predictions"].values[0])
    
    correct = [1 if true == pred else 0 for true, pred in zip(true_labels, prediction)]

    correct_predictions[user_id] = correct

In [32]:
correct_predictions

{'P1': [0, 0, 0, 1, 1, 1, 0, 1],
 'P10': [0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
 'P11': [1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1],
 'P12': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'P13': [0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
 'P14': [1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
 'P15': [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'P16': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'P17': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'P18': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 'P19': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'P2': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 'P20': [0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'P21': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'P22': [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
 'P23': [1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1],
 'P24': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
 'P25': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
 'P26': [1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0],
 'P27': [1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
 'P28': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
 'P29': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 '

In [19]:
# Find the maximum sample size across all participants
max_samples = max(len(v) for v in correct_predictions.values())

# Initialize a list to store the average correct response for each sample
average_responses = []

# Loop through each sample index
for i in range(max_samples):
    total_response = 0
    count = 0
    
    # For each participant, check if they have a response for the current sample
    for participant, responses in correct_predictions.items():
        if i < len(responses):  # Ensure the participant has data for the current sample
            total_response += responses[i]
            count += 1
    
    # Calculate the average for this sample
    if count > 0:
        average_responses.append((total_response / count)*100)
    else:
        average_responses.append(0)

# Output the average responses per sample
print(average_responses)

[66.66666666666666, 56.666666666666664, 60.0, 70.0, 70.0, 66.66666666666666, 66.66666666666666, 70.0, 64.28571428571429, 75.0, 64.28571428571429, 66.66666666666666, 75.0]


### Retrain Model if Prediction is Wrong

In [33]:
grouped_users = full_dataframe.groupby('Users')

training_set = {}

for user_id, user_data in grouped_users:
    training_set[user_id] = user_data[user_data["Condition"].str.contains(
                "Baseline") | user_data["Condition"].str.contains("PreStroop")]

In [34]:
grouped_users = sr_dataframe.groupby('User')

predictions_with_training = {}

for user_id, user_data in grouped_users:
    
    print(f'User = {user_id}')

    X_train = training_set[user_id][columns]
    Y_train = training_set[user_id]["Arousal"]
    
    X_test = np.array(user_data[columns])
    Y_test = np.array(user_data["Arousal"])
    model = joblib.load(f"./user_models/model_{user_id}.pkl")
    
    print(eval(df_users[df_users["User"] == user_id]["Predictions"].values[0]))

    new_predictions = []
    
    for i in range(len(correct_predictions[user_id])):
        x_sample = X_test[i].reshape(1, -1)
        y_sample = np.array([Y_test[i]])


        # Add the current sample to the training data
        X_train_updated = np.vstack((X_train, x_sample))  # Add new sample to training features
        Y_train_updated = np.hstack((Y_train, y_sample))
        
        print(f'Y_true = {y_sample[0]}, Prediction = {model.predict(x_sample)[0]}')

        new_predictions.append(model.predict(x_sample)[0])

        if model.predict(x_sample)[0] != y_sample[0]:

            print("Wrong Prediction, Retraining Model")
            
            # Retrain the model depending on whether it supports incremental learning
            for name, estimator in model.named_estimators_.items():
                if is_incremental_model(estimator):
                    # Use partial_fit for incremental learning models
                    if i == 0:  # partial_fit needs to know all classes in the first call
                        estimator.partial_fit(
                            x_sample, y_sample, classes=model.classes_)
                    else:
                        estimator.partial_fit(x_sample, y_sample)
                else:
                    # Re-train from scratch for non-incremental learning models
                    estimator.fit(X_train_updated, Y_train_updated)
        else:
            print("Correct Prediction")
            continue
                
    predictions_with_training[user_id] = new_predictions

User = P1
['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low']
Y_true = Medium, Prediction = Low
Wrong Prediction, Retraining Model


C:\ProgramData\Anaconda3\envs\_Loopsystem\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Y_true = High, Prediction = Medium
Wrong Prediction, Retraining Model


C:\ProgramData\Anaconda3\envs\_Loopsystem\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Y_true = Medium, Prediction = High
Wrong Prediction, Retraining Model


...

Y_true = Medium, Prediction = Medium
Correct Prediction
Y_true = Low, Prediction = Medium
Wrong Prediction, Retraining Model


C:\ProgramData\Anaconda3\envs\_Loopsystem\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Y_true = Medium, Prediction = Low
Wrong Prediction, Retraining Model


C:\ProgramData\Anaconda3\envs\_Loopsystem\lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Y_true = Medium, Prediction = Medium
Correct Prediction
Y_true = Medium, Prediction = Medium
Correct Prediction
User = P9
['Low', 'Medium', 'Low', 'Medium', 'Low', 'Low', 'Low', 'Low']
Y_true = Low, Prediction = Low
Correct Prediction
Y_true = Low, Prediction = Medium
Wrong Prediction, Retraining Model
Y_true = Low, Prediction = Low
Correct Prediction
Y_true = Low, Prediction = Low
Correct Prediction
Y_true = Low, Prediction = Low
Correct Prediction
Y_true = Low, Prediction = Low
Correct Prediction
Y_true = Low, Prediction = Low
Correct Prediction
Y_true = Low, Prediction = Low
Correct Prediction


In [35]:
predictions_with_training

{'P1': ['Low', 'Medium', 'High', 'Medium', 'Low', 'Low', 'Low', 'Medium'],
 'P10': ['High',
  'Medium',
  'High',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low'],
 'P11': ['Medium',
  'Medium',
  'Low',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium'],
 'P12': ['High',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium',
  'Medium'],
 'P13': ['High',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low'],
 'P14': ['Low',
  'Low',
  'Medium',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low'],
 'P15': ['Low',
  'Medium',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low'],
 'P16': ['Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low',
  'Low'],
 'P17': ['Low',
  'Medium',
  'Medium

In [45]:
for user in predictions_with_training:

    results = []
    
    new_accuracy = metrics.accuracy_score(eval(df_users[df_users["User"] == user]["True labels"].values[0]),predictions_with_training[user])*100
    
    print(f'User {user} previous accuracy = {df_users[df_users["User"] == user]["Accuracy"].values[0]}%.')
    print(f'User {user} new accuracy = {new_accuracy:.2f}% \n')
    
    results.append({
        "User": user,
        "True labels": eval(df_users[df_users["User"] == user]["True labels"].values[0]),
        "Predictions without Training": eval(df_users[df_users["User"] == user]["Predictions"].values[0]),
        "Predictions with Training": predictions_with_training[user],
        "Accuracy": df_users[df_users["User"] == user]["Accuracy"].values[0],
        "Accuracy with Training": new_accuracy,
    })

    pd.DataFrame(results).to_csv(f'./voting_results/results_with_training_{user}.csv', index=False)

User P1 previous accuracy = 50.0%.
User P1 new accuracy = 25.00% 

User P10 previous accuracy = 69.23076923076923%.
User P10 new accuracy = 69.23% 

User P11 previous accuracy = 83.33333333333334%.
User P11 new accuracy = 100.00% 

User P12 previous accuracy = 0.0%.
User P12 new accuracy = 90.91% 

User P13 previous accuracy = 83.33333333333334%.
User P13 new accuracy = 83.33% 

User P14 previous accuracy = 41.66666666666667%.
User P14 new accuracy = 100.00% 

User P15 previous accuracy = 91.66666666666666%.
User P15 new accuracy = 100.00% 

User P16 previous accuracy = 100.0%.
User P16 new accuracy = 100.00% 

User P17 previous accuracy = 0.0%.
User P17 new accuracy = 91.67% 

User P18 previous accuracy = 8.333333333333332%.
User P18 new accuracy = 91.67% 

User P19 previous accuracy = 100.0%.
User P19 new accuracy = 100.00% 

User P2 previous accuracy = 90.9090909090909%.
User P2 new accuracy = 90.91% 

User P20 previous accuracy = 83.33333333333334%.
User P20 new accuracy = 83.33% 


In [7]:
grouped_users = full_dataframe.groupby('Users')

dataframes_training=[]

for user_id, user_data in grouped_users:
    # Load saved results for this user
    csv_file = f'./voting_results/results_with_training_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user
    dataframes_training.append(results_df)

df_users_training = pd.concat(dataframes_training,ignore_index=True)

NameError: name 'full_dataframe' is not defined

In [42]:
df_users_training

,User,True labels,Predictions without Training,Predictions with Training,Accuracy,Accuracy with Training
0,P1,"['Medium', 'High', 'Medium', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'High', 'Medium', 'Low', 'Lo...",50.000000,25.000000
1,P10,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Medium', 'High', 'Low', 'Low', 'High...","['High', 'Medium', 'High', 'Low', 'Low', 'Low'...",69.230769,69.230769
2,P11,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...",83.333333,100.000000
3,P12,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['High', 'High', 'High', 'High', 'High', 'High...","['High', 'Medium', 'Medium', 'Medium', 'Medium...",0.000000,90.909091
4,P13,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...",83.333333,83.333333
5,P14,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",41.666667,100.000000
6,P15,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",91.666667,100.000000
7,P16,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",100.000000,100.000000
8,P17,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Medium', 'Medium', 'Medium'...",0.000000,91.666667
9,P18,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...",8.333333,91.666667


In [52]:
grouped_users = df_users_training.groupby('User')

accuracy = []
for user_id, user_data in grouped_users:
    accuracy.append(metrics.accuracy_score(eval(df_users[df_users["User"] == user_id]["True labels"].values[0]),predictions_with_training[user_id])*100)
print(accuracy)

[25.0, 76.92307692307693, 91.66666666666666, 90.9090909090909, 91.66666666666666, 91.66666666666666, 91.66666666666666, 100.0, 91.66666666666666, 91.66666666666666, 100.0, 90.9090909090909, 83.33333333333334, 100.0, 66.66666666666666, 33.33333333333333, 91.66666666666666, 69.23076923076923, 61.53846153846154, 83.33333333333334, 91.66666666666666, 100.0, 100.0, 58.333333333333336, 100.0, 75.0, 100.0, 66.66666666666666, 66.66666666666666, 87.5]


In [56]:
df_users_training["Accuracy Training when Wrong"] = accuracy

In [60]:
df_users_training

,User,True labels,Predictions without Training,Predictions with Training,Accuracy,Accuracy with Training,Accuracy Training when Wrong,Predictions with Training when Wrong
0,P1,"['Medium', 'High', 'Medium', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'High', 'Medium', 'Low', 'Lo...",50.000000,25.000000,25.000000,NaN
1,P10,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Medium', 'High', 'Low', 'Low', 'High...","['High', 'Medium', 'High', 'Low', 'Low', 'Low'...",69.230769,69.230769,76.923077,NaN
2,P11,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...",83.333333,100.000000,91.666667,NaN
3,P12,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['High', 'High', 'High', 'High', 'High', 'High...","['High', 'Medium', 'Medium', 'Medium', 'Medium...",0.000000,90.909091,90.909091,NaN
4,P13,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...",83.333333,83.333333,91.666667,NaN
5,P14,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",41.666667,100.000000,91.666667,NaN
6,P15,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",91.666667,100.000000,91.666667,NaN
7,P16,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...",100.000000,100.000000,100.000000,NaN
8,P17,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Medium', 'Medium', 'Medium'...",0.000000,91.666667,91.666667,NaN
9,P18,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...",8.333333,91.666667,91.666667,NaN


In [3]:
df_users_training = pd.read_csv(f'./voting_results/users_results_with_training.csv')
df_users_training

,User,True labels,Predictions without Training,Predictions with Training,Predictions with Training when Wrong,Predictions with SR Training,Predictions with SR Training Resample,Accuracy,Accuracy with Training,Accuracy Training when Wrong,Accuracy with SR Training,Accuracy with SR Training Resample
0,P1,"['Medium', 'High', 'Medium', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'High', 'Medium', 'Low', 'Lo...","['Low', 'Medium', 'High', 'Medium', 'Low', 'Lo...","['Low', 'Medium', 'Medium', 'Low', 'Low', 'Low...","['Low', 'Medium', 'Medium', 'High', 'Low', 'Lo...",50.000000,25.000000,25.000000,62.500000,50.000000
1,P10,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Medium', 'High', 'Low', 'Low', 'High...","['High', 'Medium', 'High', 'Low', 'Low', 'Low'...","['High', 'Medium', 'High', 'Low', 'Low', 'Low'...","['Low', 'Low', 'High', 'High', 'Low', 'Low', '...","['Low', 'Low', 'High', 'High', 'Low', 'Low', '...",69.230769,69.230769,76.923077,76.923077,76.923077
2,P11,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...",83.333333,100.000000,91.666667,75.000000,75.000000
3,P12,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['High', 'High', 'High', 'High', 'High', 'High...","['High', 'Medium', 'Medium', 'Medium', 'Medium...","['High', 'Medium', 'Medium', 'Medium', 'Medium...","['Low', 'Low', 'High', 'Medium', 'Low', 'Mediu...","['Low', 'Low', 'High', 'Medium', 'Medium', 'Me...",0.000000,90.909091,90.909091,63.636364,63.636364
4,P13,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...",83.333333,83.333333,91.666667,91.666667,83.333333
5,P14,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...",41.666667,100.000000,91.666667,91.666667,91.666667
6,P15,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...",91.666667,100.000000,91.666667,91.666667,91.666667
7,P16,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Low', 'Low', 'Low', 'Med...","['Medium', 'Medium', 'Low', 'Low', 'Low', 'Med...",100.000000,100.000000,100.000000,75.000000,75.000000
8,P17,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Medium', 'Medium', 'Medium'...","['Low', 'Medium', 'Medium', 'Medium', 'Medium'...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...",0.000000,91.666667,91.666667,91.666667,91.666667
9,P18,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...",8.333333,91.666667,91.666667,91.666667,91.666667


In [12]:
grouped_users = df_users_training.groupby('User')

dataframes_training=[]

for user_id, user_data in grouped_users:
    # Load saved results for this user
    csv_file = f'./voting_results/SRtraining_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user
    dataframes_training.append(results_df)

SR_training = pd.concat(dataframes_training,ignore_index=True)

In [6]:
grouped_users = df_users_training.groupby('User')

dataframes_training=[]

for user_id, user_data in grouped_users:
    # Load saved results for this user
    csv_file = f'./voting_results/SRtraining_resampling_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user
    dataframes_training.append(results_df)

SR_training_resample = pd.concat(dataframes_training,ignore_index=True)

In [37]:
df_users_training["Predictions with SR Training"] = SR_training["Predictions with SR Training"]
df_users_training["Predictions with SR Training Resample"] = SR_training_resample["Predictions with SR Training"]

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_7136\4201379636.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_training["Predictions with SR Training"] = SR_training["Predictions with SR Training"]
C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_7136\4201379636.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_training["Predictions with SR Training Resample"] = SR_training_resample["Predictions with SR Training"]


In [15]:
df_users_training = df_users_training.iloc[:, [0, 1, 2, 3, 7, 8, 9, 4, 5, 6]]

In [38]:
grouped_users = df_users_training.groupby('User')

accuracy = []
for user_id, user_data in grouped_users:
    accuracy.append(metrics.accuracy_score(eval(df_users_training[df_users_training["User"] == user_id]["True labels"].values[0]),eval(df_users_training[df_users_training["User"] == user_id]["Predictions with SR Training"].values[0]))*100)
print(accuracy)
df_users_training["Accuracy with SR Training"] = accuracy

[62.5, 76.92307692307693, 66.66666666666666, 63.63636363636363, 91.66666666666666, 91.66666666666666, 91.66666666666666, 75.0, 91.66666666666666, 91.66666666666666, 91.66666666666666, 90.9090909090909, 58.333333333333336, 81.81818181818183, 41.66666666666667, 58.333333333333336, 91.66666666666666, 53.84615384615385, 46.15384615384615, 75.0, 58.333333333333336, 83.33333333333334, 66.66666666666666, 58.333333333333336, 92.3076923076923, 66.66666666666666, 63.63636363636363, 75.0, 58.333333333333336, 87.5]


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_7136\2636561442.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_training["Accuracy with SR Training"] = accuracy


In [21]:
grouped_users = df_users_training.groupby('User')

accuracy = []
for user_id, user_data in grouped_users:
    accuracy.append(metrics.accuracy_score(eval(df_users_training[df_users_training["User"] == user_id]["True labels"].values[0]),eval(df_users_training[df_users_training["User"] == user_id]["Predictions with SR Training Resample"].values[0]))*100)
print(accuracy)
df_users_training["Accuracy with SR Training Resample"] = accuracy

[50.0, 76.92307692307693, 75.0, 63.63636363636363, 83.33333333333334, 91.66666666666666, 91.66666666666666, 75.0, 91.66666666666666, 91.66666666666666, 100.0, 90.9090909090909, 50.0, 81.81818181818183, 33.33333333333333, 58.333333333333336, 83.33333333333334, 61.53846153846154, 38.46153846153847, 75.0, 58.333333333333336, 83.33333333333334, 66.66666666666666, 58.333333333333336, 92.3076923076923, 66.66666666666666, 63.63636363636363, 75.0, 58.333333333333336, 87.5]


C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_7136\834892345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_users_training["Accuracy with SR Training Resample"] = accuracy


In [39]:
df_users_training

,User,True labels,Predictions without Training,Predictions with Training,Predictions with Training when Wrong,Predictions with SR Training,Predictions with SR Training Resample,Accuracy,Accuracy with Training,Accuracy Training when Wrong,Accuracy with SR Training,Accuracy with SR Training Resample
0,P1,"['Medium', 'High', 'Medium', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'High', 'Medium', 'Low', 'Lo...","['Low', 'Medium', 'High', 'Medium', 'Low', 'Lo...","['Low', 'Medium', 'Medium', 'Low', 'Low', 'Low...","['Low', 'Medium', 'Medium', 'High', 'Low', 'Lo...",50.000000,25.000000,25.000000,62.500000,50.000000
1,P10,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Medium', 'High', 'Low', 'Low', 'High...","['High', 'Medium', 'High', 'Low', 'Low', 'Low'...","['High', 'Medium', 'High', 'Low', 'Low', 'Low'...","['Low', 'Low', 'High', 'High', 'Low', 'Low', '...","['Low', 'Low', 'High', 'High', 'Low', 'Low', '...",69.230769,69.230769,76.923077,76.923077,76.923077
2,P11,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Low', 'Medium', 'Medium'...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...",83.333333,100.000000,91.666667,66.666667,75.000000
3,P12,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['High', 'High', 'High', 'High', 'High', 'High...","['High', 'Medium', 'Medium', 'Medium', 'Medium...","['High', 'Medium', 'Medium', 'Medium', 'Medium...","['Low', 'Low', 'High', 'Low', 'Medium', 'Mediu...","['Low', 'Low', 'High', 'Medium', 'Medium', 'Me...",0.000000,90.909091,90.909091,63.636364,63.636364
4,P13,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...","['High', 'Low', 'Low', 'Low', 'Low', 'Low', 'L...",83.333333,83.333333,91.666667,91.666667,83.333333
5,P14,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Medium', 'Medium', '...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Medium', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...",41.666667,100.000000,91.666667,91.666667,91.666667
6,P15,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...","['Low', 'Medium', 'Low', 'Low', 'Low', 'Low', ...",91.666667,100.000000,91.666667,91.666667,91.666667
7,P16,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Low', 'Low', 'Low', 'Low...","['Medium', 'Medium', 'Low', 'Low', 'Low', 'Med...",100.000000,100.000000,100.000000,75.000000,75.000000
8,P17,"['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Low', 'Medium', 'Medium', 'Medium', 'Medium'...","['Low', 'Medium', 'Medium', 'Medium', 'Medium'...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...",0.000000,91.666667,91.666667,91.666667,91.666667
9,P18,"['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Lo...","['Medium', 'Medium', 'Medium', 'Medium', 'Medi...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...","['Medium', 'Low', 'Low', 'Low', 'Low', 'Low', ...",8.333333,91.666667,91.666667,91.666667,91.666667


In [23]:
df_users_training.to_csv(f'./voting_results/users_results_with_training.csv', index=False)

### Average across participants

In [4]:
grouped_users = df_users_training.groupby('User')

notraining_predictions, training_predictions, training_wrong_predictions, predictions_with_sr, predictions_sr_resample = {}, {}, {}, {},{}

for user_id, user_data in grouped_users:
    true_labels = eval(user_data["True labels"].values[0])
    prediction_notraining = eval(user_data["Predictions without Training"].values[0])
    prediction_training = eval(user_data["Predictions with Training"].values[0])
    predictions_when_wrong = eval(user_data["Predictions with Training when Wrong"].values[0])
    sr_predictions = eval(user_data["Predictions with SR Training"].values[0])
    sr_predictions_resample =  eval(user_data["Predictions with SR Training Resample"].values[0])
    
    notraining_predictions[user_id] = [1 if true == pred else 0 for true, pred in zip(true_labels, prediction_notraining)]
    training_predictions[user_id] = [1 if true == pred else 0 for true, pred in zip(true_labels, prediction_training)]
    training_wrong_predictions[user_id] = [1 if true == pred else 0 for true, pred in zip(true_labels, predictions_when_wrong)]
    predictions_with_sr[user_id] = [1 if true == pred else 0 for true, pred in zip(true_labels, sr_predictions)]
    predictions_sr_resample[user_id] = [1 if true == pred else 0 for true, pred in zip(true_labels, sr_predictions_resample)]

In [5]:
# Find the maximum sample size across all participants
max_samples = max(len(v) for v in notraining_predictions.values())

# Initialize a list to store the average correct response for each sample
notraining_responses = []

# Loop through each sample index
for i in range(max_samples):
    total_response = 0
    count = 0
    
    # For each participant, check if they have a response for the current sample
    for participant, responses in notraining_predictions.items():
        if i < len(responses):  # Ensure the participant has data for the current sample
            total_response += responses[i]
            count += 1
    
    # Calculate the average for this sample
    if count > 0:
        notraining_responses.append((total_response / count)*100)
    else:
        notraining_responses.append(0)

# Output the average responses per sample
print(notraining_responses)

[66.66666666666666, 56.666666666666664, 60.0, 70.0, 70.0, 66.66666666666666, 66.66666666666666, 70.0, 64.28571428571429, 75.0, 64.28571428571429, 66.66666666666666, 75.0]


In [6]:
# Find the maximum sample size across all participants
max_samples = max(len(v) for v in training_predictions.values())

# Initialize a list to store the average correct response for each sample
training_responses = []

# Loop through each sample index
for i in range(max_samples):
    total_response = 0
    count = 0
    
    # For each participant, check if they have a response for the current sample
    for participant, responses in training_predictions.items():
        if i < len(responses):  # Ensure the participant has data for the current sample
            total_response += responses[i]
            count += 1
    
    # Calculate the average for this sample
    if count > 0:
        training_responses.append((total_response / count)*100)
    else:
        training_responses.append(0)

# Output the average responses per sample
print(training_responses)

[66.66666666666666, 83.33333333333334, 76.66666666666667, 86.66666666666667, 90.0, 93.33333333333333, 80.0, 80.0, 78.57142857142857, 85.71428571428571, 78.57142857142857, 87.5, 100.0]


In [7]:
# Find the maximum sample size across all participants
max_samples = max(len(v) for v in training_wrong_predictions.values())

# Initialize a list to store the average correct response for each sample
training_when_wrong_responses = []

# Loop through each sample index
for i in range(max_samples):
    total_response = 0
    count = 0
    
    # For each participant, check if they have a response for the current sample
    for participant, responses in training_wrong_predictions.items():
        if i < len(responses):  # Ensure the participant has data for the current sample
            total_response += responses[i]
            count += 1
    
    # Calculate the average for this sample
    if count > 0:
        training_when_wrong_responses.append((total_response / count)*100)
    else:
        training_when_wrong_responses.append(0)

# Output the average responses per sample
print(training_when_wrong_responses)

[66.66666666666666, 80.0, 70.0, 86.66666666666667, 90.0, 96.66666666666667, 80.0, 83.33333333333334, 82.14285714285714, 89.28571428571429, 82.14285714285714, 83.33333333333334, 100.0]


In [8]:
# Find the maximum sample size across all participants
max_samples = max(len(v) for v in predictions_with_sr.values())

# Initialize a list to store the average correct response for each sample
training_sr_responses = []

# Loop through each sample index
for i in range(max_samples):
    total_response = 0
    count = 0
    
    # For each participant, check if they have a response for the current sample
    for participant, responses in predictions_with_sr.items():
        if i < len(responses):  # Ensure the participant has data for the current sample
            total_response += responses[i]
            count += 1
    
    # Calculate the average for this sample
    if count > 0:
        training_sr_responses.append((total_response / count)*100)
    else:
        training_sr_responses.append(0)

# Output the average responses per sample
print(training_sr_responses)

[26.666666666666668, 63.33333333333333, 80.0, 80.0, 76.66666666666667, 83.33333333333334, 76.66666666666667, 83.33333333333334, 78.57142857142857, 82.14285714285714, 75.0, 70.83333333333334, 75.0]


In [9]:
# Find the maximum sample size across all participants
max_samples = max(len(v) for v in predictions_sr_resample.values())

# Initialize a list to store the average correct response for each sample
training_sr_resample_responses = []

# Loop through each sample index
for i in range(max_samples):
    total_response = 0
    count = 0
    
    # For each participant, check if they have a response for the current sample
    for participant, responses in predictions_sr_resample.items():
        if i < len(responses):  # Ensure the participant has data for the current sample
            total_response += responses[i]
            count += 1
    
    # Calculate the average for this sample
    if count > 0:
        training_sr_resample_responses.append((total_response / count)*100)
    else:
        training_sr_resample_responses.append(0)

# Output the average responses per sample
print(training_sr_resample_responses)

[26.666666666666668, 60.0, 83.33333333333334, 76.66666666666667, 80.0, 80.0, 76.66666666666667, 83.33333333333334, 78.57142857142857, 78.57142857142857, 67.85714285714286, 79.16666666666666, 75.0]


###  Incremental Accuracy No Training

In [14]:
import warnings
warnings.filterwarnings("ignore")
users_dict = {}

grouped_users = full_dataframe.groupby('Users')

for user_id, user_data in grouped_users:
        
    # Load saved results for this user
    csv_file = f'results_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user

    print("User = {}".format(user_id))
    
    accuracy_scores = {}
    
    results, user_accuracy, user_precision, user_f1_score, user_recall,  samples = [
    ], [], [], [], [], []

        training_data = user_data[user_data["Condition"].str.contains(
            "Baseline") | user_data["Condition"].str.contains("PreStroop")]
    testing_data = user_data[user_data["Condition"].str.contains(
        "VR")] #| user_data["Condition"].str.contains("PostStroop")]

    X_train = np.array(training_data[columns])
    X_test = np.array(testing_data[columns])
    Y_train = np.array(training_data["Arousal"])
    Y_test = np.array(testing_data["Arousal"])

    # Encode the target labels (Y_train and Y_test)
    # Y_train = label_encoder.fit_transform(Y_train)
    # Y_test = label_encoder.transform(Y_test)

    imp = SimpleImputer(missing_values=np.nan, strategy="mean")
    scaler = StandardScaler()

    imp = imp.fit(X_train)

    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)

    scaler = scaler.fit(X_train, Y_train.ravel())
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    rfe = feature_selection.RFE(ensemble.RandomForestClassifier(random_state=42), step=1)
    rfe = rfe.fit(X_train, Y_train.ravel())
    X_train = rfe.transform(X_train)
    X_test = rfe.transform(X_test)

    #Train each possible model on the training data and calculate accuracy score
    
    for model_name,model in models:
        
        # print("Model = " + model_name)
        
        user_results = results_df[(results_df['Model'] == model_name)]
        
        best_params = eval(user_results['Best Parameters'].values[0])  # Convert string back to dictionary
        best_params = {key.replace('model__', ''): value for key, value in best_params.items()} #remove 'model__' prefix
        
        model.set_params(**best_params)

        model.fit(X_train, Y_train.ravel())
    
        model_y_predict = model.predict(X_test)
    
        accuracy = metrics.accuracy_score(Y_test, model_y_predict)

        accuracy_scores[model_name] = accuracy*100

    # Select the two best models
    sorted_models = sorted(accuracy_scores.items(),
                           key=lambda item: item[1], reverse=True)
    best_two_models = sorted_models[:2]

    print(best_two_models)

    best_models = [model for model in models if model[0]
                   in [best_two_models[0][0], best_two_models[1][0]]]

    voting_clf = ensemble.VotingClassifier(
        estimators=best_models, voting='soft', n_jobs=-1)

    voting_clf.fit(X_train, Y_train.ravel())

    new_data_batch = 1
    
    y_true, y_predict = [], []
        
    for i in range(0, len(testing_data), new_data_batch):
        
        new_X_test = np.array(testing_data.iloc[i:i + new_data_batch][columns])
        new_Y_test = np.array(testing_data.iloc[i:i + new_data_batch]["Arousal"])
        
        new_Y_test = encoder.transform(new_Y_test)

        new_X_test = imp.transform(new_X_test)
        new_X_test = scaler.transform(new_X_test)
        new_X_test = rfe.transform(new_X_test)

        y_true.extend(encoder.inverse_transform(new_Y_test))
        y_predict.extend(encoder.inverse_transform(
            voting_clf.predict(new_X_test)))

        accuracy = metrics.accuracy_score(y_true, y_predict)
        
        precision = metrics.precision_score(
            y_true, y_predict, average="weighted", zero_division=1)
        recall = metrics.recall_score(y_true, y_predict, average='weighted')
        f1_score = metrics.f1_score(
            Y_test, model.predict(X_test), average="weighted")

        user_accuracy.append(accuracy*100)
        user_precision.append(precision*100)
        user_f1_score.append(f1_score*100)
        user_recall.append(recall*100)
        samples.append(i)

    results.append({
            "User": user_id,
            "True labels": y_true,
            "Predictions": y_predict,
            "Accuracy": user_accuracy,
            "Precision": user_precision,
            "F1-Score": user_f1_score,
            "Recall": user_recall
        })

    pd.DataFrame(results).to_csv(
        f'./incremental_notraining_step1/results_{user_id}.csv', index=False)
    
    print(f'True Labels = {y_true}')
    print(f'Predicted Labels = {y_predict}')

User = P1
[('SVM', 53.09597523219814), ('KNN', 49.07120743034056)]
True Labels = ['Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'Medium', 'High', 'High', 'High', 'High', 'High', 'High', 

[('Random Forest', 81.84110970996217), ('Bayes', 72.50945775535939)]
True Labels = ['Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low', 'Low',

KeyboardInterrupt: 

### Incremental Accuracy Training

In [32]:
import warnings
warnings.filterwarnings("ignore")
users_dict = {}

grouped_users = full_dataframe.groupby('Users')

for user_id, user_data in grouped_users:

    # Load saved results for this user
    csv_file = f'./user_results/results_{user_id}.csv'
    try:
        results_df = pd.read_csv(csv_file)
    except FileNotFoundError:
        results_df = None  # No saved results for this user

    print("User = {}".format(user_id))

    encoder = LabelEncoder()

    results, user_accuracy, user_precision, user_f1_score, user_recall,  samples = [
    ], [], [], [], [], []

        training_data = user_data[user_data["Condition"].str.contains(
            "Baseline") | user_data["Condition"].str.contains("PreStroop")]
    testing_data = user_data[user_data["Condition"].str.contains(
        "VR")] #| user_data["Condition"].str.contains("PostStroop")]

    X_train = np.array(training_data[columns])
    X_test = np.array(testing_data[columns])
    Y_train = np.array(training_data["Arousal"])
    Y_test = np.array(testing_data["Arousal"])

    # Encode the target labels (Y_train and Y_test)
    # Y_train = label_encoder.fit_transform(Y_train)
    # Y_test = label_encoder.transform(Y_test)

    imp = SimpleImputer(missing_values=np.nan, strategy="mean")
    scaler = StandardScaler()

    imp = imp.fit(X_train)

    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)

    scaler = scaler.fit(X_train, Y_train.ravel())
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    rfe = feature_selection.RFE(ensemble.RandomForestClassifier(random_state=42), step=1)
    rfe = rfe.fit(X_train, Y_train.ravel())
    X_train = rfe.transform(X_train)
    X_test = rfe.transform(X_test)

    #Train each possible model on the training data and calculate accuracy score
    
    for model_name,model in models:
        
        # print("Model = " + model_name)
        
        user_results = results_df[(results_df['Model'] == model_name)]
        
        best_params = eval(user_results['Best Parameters'].values[0])  # Convert string back to dictionary
        best_params = {key.replace('model__', ''): value for key, value in best_params.items()} #remove 'model__' prefix
        
        model.set_params(**best_params)

        model.fit(X_train, Y_train.ravel())
    
        model_y_predict = model.predict(X_test)
    
        accuracy = metrics.accuracy_score(Y_test, model_y_predict)

        accuracy_scores[model_name] = accuracy*100

    # Select the two best models
    sorted_models = sorted(accuracy_scores.items(),
                           key=lambda item: item[1], reverse=True)
    best_two_models = sorted_models[:2]

    print(best_two_models)

    best_models = [model for model in models if model[0]
                   in [best_two_models[0][0], best_two_models[1][0]]]

    voting_clf = ensemble.VotingClassifier(
        estimators=best_models, voting='soft', n_jobs=-1)

    voting_clf.fit(X_train, Y_train.ravel())

    new_data_batch = 1
    
    true_labels, predicted_labels = [], []
    
    # Initialize a variable to keep track of the offset due to shrinking test set
    index_offset = 0
    
    number_of_samples = len(testing_data)
    
    sample_predictions = {}
    
    for i in range(len(testing_data)):
        sample_predictions[i] = {"True labels": np.array(testing_data["Arousal"])[i],
                                "Predicted_labels": []}
    
    for i in range(0, len(testing_data), new_data_batch):

        y_true, y_predict = [], []
        
        new_X_test = np.array(testing_data[columns])
        new_Y_test = np.array(testing_data["Arousal"])

        new_Y_test = encoder.transform(new_Y_test)

        new_X_test = imp.transform(new_X_test)
        new_X_test = scaler.transform(new_X_test)
        new_X_test = rfe.transform(new_X_test)
        
        indexes = list(range(index_offset,number_of_samples))
        
        for index in indexes:
            sample_predictions[index]["Predicted_labels"].append(encoder.inverse_transform(
            voting_clf.predict(new_X_test))[index-index_offset])
        
        y_true.extend(encoder.inverse_transform(new_Y_test))
        y_predict.extend(encoder.inverse_transform(
            voting_clf.predict(new_X_test)))

        accuracy = metrics.accuracy_score(y_true, y_predict)

#         print(f'Accuracy = {accuracy*100:.2f}%')

        precision = metrics.precision_score(
            y_true, y_predict, average="weighted", zero_division=1)
        recall = metrics.recall_score(y_true, y_predict, average='weighted')
        f1_score = metrics.f1_score(
            Y_test, model.predict(X_test), average="weighted")

        user_accuracy.append(accuracy*100)
        user_precision.append(precision*100)
        user_f1_score.append(f1_score*100)
        user_recall.append(recall*100)
        samples.append(i)

        # Select one sample from the test set (e.g., the first one)
        selected_sample_X = X_test[i]
        selected_sample_Y = Y_test[i]

        # Add the current batch to the training set
        X_train = np.vstack([X_train, selected_sample_X])
        Y_train = np.hstack([Y_train, selected_sample_Y])

        # Remove the selected sample from the test set
        testing_data = testing_data.drop(
            testing_data.index[0]).reset_index(drop=True)

        # Increment the offset to account for the removed samples
        index_offset += new_data_batch
        
        # Retrain the model depending on whether it supports incremental learning
        for name, estimator in voting_clf.named_estimators_.items():
            if is_incremental_model(estimator):
                # Use partial_fit for incremental learning models
                if i == 0:  # partial_fit needs to know all classes in the first call
                    estimator.partial_fit(
                        X_train, Y_train.ravel(), classes=np.unique(Y_train))
                else:
                    estimator.partial_fit(X_train, Y_train.ravel())
            else:
                # Re-train from scratch for non-incremental learning models
                estimator.fit(X_train, Y_train.ravel())

    results.append({
        "User": user_id,
#         "True labels": true_labels,
#         "Predictions": predicted_labels,
        "Sample Predictions": sample_predictions,
        "Accuracy": user_accuracy,
        "Precision": user_precision,
        "F1-Score": user_f1_score,
        "Recall": user_recall
    })

    pd.DataFrame(results).to_csv(
        f'./incremental_training_step1/results_{user_id}.csv', index=False)

User = P1
[('SVM', 53.09597523219814), ('KNN', 49.07120743034056)]
User = P10
[('Random Forest', 81.84110970996217), ('Bayes', 72.50945775535939)]
User = P11
[('Logistic Regression', 13.696808510638297), ('NN', 13.164893617021276)]
User = P12
[('Random Forest', 1.3227513227513228), ('SVM', 0.9259259259259258)]
User = P13
[('NN', 92.14659685863874), ('KNN', 85.20942408376963)]
User = P14
[('Random Forest', 89.25831202046037), ('NN', 34.271099744245525)]
User = P15
[('Random Forest', 99.86962190352021), ('SVM', 98.82659713168188)]
User = P16
[('SVM', 29.161118508655125), ('Bayes', 0.2663115845539281)]
User = P17
[('Random Forest', 70.97625329815304), ('Bayes', 66.35883905013192)]
User = P18
[('KNN', 58.322411533420706), ('SVM', 24.770642201834864)]
User = P19
[('Bayes', 88.69123252858958), ('KNN', 87.67471410419314)]
User = P2
[('Random Forest', 98.27814569536424), ('SVM', 97.88079470198674)]
User = P20
[('Random Forest', 77.83711615487317), ('Bayes', 61.01468624833111)]
User = P21
[('Ba

## Save to CSV

In [19]:
import csv
# Open a CSV file for writing
with open('voting_best2_trained.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['user_id','samples','accuracies'])
    
    # Write the data
    for user_id, data in users_dict.items():
        # Join accuracies into a single string
        row = [user_id, ', '.join(map(str, data['Samples'])),', '.join(map(str, data['Accuracy']))]
        writer.writerow(row)

In [22]:
import csv
with open('models_accuracies_trained.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['user_id', 'accuracies'])
    
    # Write the data
    for user_id, data in users_dict.items():
        for model, accuracies in data.items():
            # Join accuracies into a single string
            row = [user_id + "_" + model, ', '.join(map(str, accuracies))]
            writer.writerow(row)

In [ ]:
import csv
# Open a CSV file for writing
with open('soft_voting.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(['user_id', 'accuracies'])
    
    # Write the data
    for user_id, accuracies in users_dict.items():
        # Join accuracies into a single string
        row = [user_id, ', '.join(map(str, accuracies))]
        writer.writerow(row)

## Read CSV

Load Each model accuracy

In [ ]:
import csv
# Read from the CSV file

# Initialize an empty dictionary to store the accuracies
accuracy_dict = {}

with open('decrease_models_accuracies.csv', mode='r') as file:
    reader = csv.reader(file)
    
    # Skip the header
    header = next(reader)
    
    for row in reader:
        user_id = row[0].split('_')[0]  # Extract the user ID (e.g., 'P1')
        model = row[0].split('_')[1]     # Extract the model name (e.g., 'KNN')
        
        # Split the accuracies string back into a list of floats
        accuracies = list(map(float, row[1].split(', '))) if row[1] else []
        
        # Create nested dictionary structure
        if user_id not in accuracy_dict:
            accuracy_dict[user_id] = {}
        
        accuracy_dict[user_id][model] = accuracies

In [5]:
import csv
# Read from the CSV file

# Initialize an empty dictionary to store the accuracies
accuracy_dict = {}

with open('models_accuracies.csv', mode='r') as file:
    reader = csv.reader(file)
    
    # Skip the header
    header = next(reader)
    
    for row in reader:
        user_id = row[0].split('_')[0]  # Extract the user ID (e.g., 'P1')
        model = row[0].split('_')[1]     # Extract the model name (e.g., 'KNN')
        
        # Split the accuracies string back into a list of floats
        accuracies = list(map(float, row[1].split(', '))) if row[1] else []
        
        # Create nested dictionary structure
        if user_id not in accuracy_dict:
            accuracy_dict[user_id] = {}
        
        accuracy_dict[user_id][model] = accuracies

In [ ]:
len(accuracy_dict["P1"]["KNN"]

Load Soft Voting Accuracy

In [4]:
import csv
soft_voting = {}

# Open a CSV file for writing
with open('soft_voting.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the header
    writer.writerow(['user_id', 'accuracies'])
    
    # Write the data
    for user_id, accuracies in accuracy_dict.items():
        # Join accuracies into a single string
        row = [user_id, ', '.join(map(str, accuracies))]
        writer.writerow(row)

NameError: name 'accuracy_dict' is not defined

In [ ]:
soft_voting

In [ ]:
accuracy_dict

## Plot Accuracy

In [31]:
%matplotlib qt
from matplotlib.legend_handler import HandlerLine2D

plt.figure(figsize=(12,12))

line1, = plt.plot(notraining_responses, marker="o", markersize=18, label="No training")
line2, = plt.plot(training_responses, marker="o", markersize=18, label="Training at each SR")
line3, = plt.plot(training_when_wrong_responses, marker="o", markersize=18, label="Training at a wrong prediction")

plt.xlabel('Time (min)', fontsize=40)
plt.ylabel('Model Accuracy (%)', fontsize=40)
# plt.title("Average Accuracy using Self-Reports", fontsize=32)
plt.xticks(np.arange(0, 13, 1),fontsize=32,rotation=45)
plt.yticks(np.arange(0, 110, 5),fontsize=32)
plt.ylim(50, 100)  # Set y-axis limits from 0 to 100%
plt.grid(axis='y', linestyle='--')

# Custom legend with smaller marker size
legend = plt.legend(fontsize=24)

# Reduce marker size in the legend
for handle in legend.legend_handles:
    handle.set_markersize(12)  # Adjust marker size for legend only

plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [41]:
%matplotlib qt
from matplotlib.legend_handler import HandlerLine2D

for user in notraining_predictions:
    plt.figure(figsize=(12,4))
    line1, = plt.plot(notraining_predictions[user], marker="o",  markersize=18, label="No training")
    line2, = plt.plot(training_wrong_predictions[user],  marker="o",  markersize=18, label="Training at a wrong prediction")

    plt.xlabel('Time (min)', fontsize=40)
    plt.ylabel('Prediction', fontsize=40)
    plt.title(f'{user}' , fontsize=40)
    plt.xticks(fontsize=32,rotation=45)
    plt.yticks([0, 1], ["Wrong", "Correct"], fontsize=32)
    plt.ylim(-0.1, 1.1)  # Set y-axis limits from 0 to 100%
    plt.grid(axis='y', linestyle='--')
    
    # Custom legend with smaller marker size
    legend = plt.legend(loc="lower center", bbox_to_anchor=(0.87, 1.05), fontsize=18)
    
    # Reduce marker size in the legend
    for handle in legend.legend_handles:
        handle.set_markersize(12)  # Adjust marker size for legend only
        
    plt.tight_layout()  # Adjust layout to prevent clipping of labels
    plt.show()

C:\Users\Rodrigo\AppData\Local\Temp\ipykernel_23496\2789844365.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(12,4))
